In [1]:
# install required packages
%pip install torch>=2.0.0 torchvision>=0.15.0 scikit-learn>=1.0 numpy>=1.21 Pillow>=9.0 argparse>=1.4.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# importing required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from PIL import Image
import numpy as np
import random

In [2]:
# setting paths and seeding
BASE_DIR = os.getcwd()
TRAIN_DIR = os.path.join(BASE_DIR, 'covid_dataset', 'train')
EVAL_DIR = os.path.join(BASE_DIR, 'evaluation_Set')
RESULT_FILE = os.path.join(BASE_DIR, 'result.txt')

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# check if required directories exist
if not os.path.exists(TRAIN_DIR):
    raise FileNotFoundError(f"Training directory not found: {TRAIN_DIR}")
if not os.path.exists(EVAL_DIR):
    raise FileNotFoundError(f"Evaluation directory not found: {EVAL_DIR}")

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) 
])

# load training dataset
train_data = datasets.ImageFolder(TRAIN_DIR, transform=transform)

# stratified split
labels = [label for _, label in train_data]
train_indices, val_indices = train_test_split(
    range(len(train_data)),
    test_size=0.2, 
    stratify=labels,  
    random_state=42  
)

# subset datasets
train_dataset = torch.utils.data.Subset(train_data, train_indices)
val_dataset = torch.utils.data.Subset(train_data, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# load evaluation dataset
def load_eval_images(eval_dir):
    return [os.path.join(eval_dir, fname) for fname in os.listdir(eval_dir) if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]

eval_image_paths = load_eval_images(EVAL_DIR)

class EvalDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, os.path.basename(img_path)

eval_dataset = EvalDataset(eval_image_paths, transform)
eval_loader = DataLoader(eval_dataset, batch_size=1, shuffle=False)

# model: ResNet18
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# training function
def train_model(epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# validation function
def validate_model():
    model.eval()
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())

    unique, counts = np.unique(val_labels, return_counts=True)
    print("Class distribution in validation set:", dict(zip(unique, counts)))

    print("True labels:", val_labels)
    print("Predicted labels:", val_preds)

    # calculate F1 score
    val_f1 = f1_score(val_labels, val_preds)
    print(f"Validation F1 Score (basic): {val_f1 * 100:.4f}%")

    # weighted F1 score
    val_f1_weighted = f1_score(val_labels, val_preds, average='weighted')
    print(f"Validation Weighted F1 Score: {val_f1_weighted:.4f}")

    return val_f1

# evaluation function
def evaluate_model():
    model.eval()
    with open(RESULT_FILE, 'w') as f:
        with torch.no_grad():
            for inputs, img_names in eval_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                img_name = img_names[0]  
                label = preds.item()
                f.write(f"{img_name:<20}{label}\n")

    print(f"Evaluation completed. Results saved to {RESULT_FILE}")

# main execution
epochs = 10
if __name__ == "__main__":
    epochs = 10

    print("Starting training...")
    train_model(epochs)
    print("Training completed.")

    print("Validating model...")
    validate_model()

    print("Evaluating model on unseen data...")
    evaluate_model()

Starting training...
Epoch 1/10, Loss: 0.6160
Epoch 2/10, Loss: 0.3063
Epoch 3/10, Loss: 0.1090
Epoch 4/10, Loss: 0.0602
Epoch 5/10, Loss: 0.0384
Epoch 6/10, Loss: 0.0482
Epoch 7/10, Loss: 0.1824
Epoch 8/10, Loss: 0.0230
Epoch 9/10, Loss: 0.0318
Epoch 10/10, Loss: 0.0204
Training completed.
Validating model...
Class distribution in validation set: {np.int64(0): np.int64(28), np.int64(1): np.int64(23)}
True labels: [np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(